## 딥러닝과 통계모델을 이용한 T-커머스 매출 예측

### Process
1. Sales record(실적 데이터)
2. weighted avverage of hourly product sales
3. sparsity control by Gaussian smoothing along time
4. Sparsity control by SVD
5. Decay by last sales day


In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def drop_unnamed(data):
    names = data.columns
    for name in names:
        if "Unnamed" in name:
            data.pop(name)
    return data

### Load Performance data

Dtypes check

In [66]:
pf = pd.read_csv("/home/yeeunlee/bigcon2020_parrot/prep/data/final_performance_v2.csv")
pf = drop_unnamed(pf)
pf.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,hour,prime,real_date,설명,IsHoliday,지속휴일수,TEMP,HUM,weekofyear
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01 00:00:00,1,06:00:00,6,0,2019-01-01 00:00:00,새해,1.0,1.0,-6.576974,63.524958,1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01 00:00:00,1,06:00:00,6,0,2019-01-01 00:00:00,새해,1.0,1.0,-6.576974,63.524958,1


In [4]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35379 entries, 0 to 35378
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   방송일시        35379 non-null  object 
 1   노출(분)       35379 non-null  float64
 2   마더코드        35379 non-null  int64  
 3   상품코드        35379 non-null  int64  
 4   상품명         35379 non-null  object 
 5   상품군         35379 non-null  object 
 6   판매단가        35379 non-null  int64  
 7   취급액         35379 non-null  float64
 8   date        35379 non-null  object 
 9   week        35379 non-null  int64  
 10  time        35379 non-null  object 
 11  hour        35379 non-null  int64  
 12  prime       35379 non-null  int64  
 13  real_date   35379 non-null  object 
 14  설명          1672 non-null   object 
 15  IsHoliday   35379 non-null  float64
 16  지속휴일수       35379 non-null  float64
 17  TEMP        35379 non-null  float64
 18  HUM         35379 non-null  float64
 19  weekofyear  35379 non-nul

In [5]:
stage = pd.read_csv("/home/yeeunlee/bigcon2020_parrot/prep/stage_1.csv")
stage.head()

,방송일시,date,상품코드,노출(분),stage
0,2019-01-01 06:00:00,2019-01-01 00:00:00,201072,20.0,early
1,2019-01-01 06:20:00,2019-01-01 00:00:00,201072,20.0,mid
2,2019-01-01 06:40:00,2019-01-01 00:00:00,201072,20.0,late
3,2019-01-01 06:00:00,2019-01-01 00:00:00,201079,20.0,early
4,2019-01-01 06:20:00,2019-01-01 00:00:00,201079,20.0,mid


In [6]:
pf = pf.merge(stage[['방송일시', '상품코드', 'stage']], on = ['방송일시', '상품코드'])
pf.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,...,hour,prime,real_date,설명,IsHoliday,지속휴일수,TEMP,HUM,weekofyear,stage
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01 00:00:00,1,...,6,0,2019-01-01 00:00:00,새해,1.0,1.0,-6.576974,63.524958,1,early
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01 00:00:00,1,...,6,0,2019-01-01 00:00:00,새해,1.0,1.0,-6.576974,63.524958,1,early


In [7]:
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import gc

In [20]:
pf_train, pf_valid = train_test_split(pf, test_size = 0.1, random_state = 123457)
print(pf_train.shape)
print(pf_valid.shape)

(31841, 21)
(3538, 21)


### Encoding

timestamp는 object type으로 뜨지만 encoding으로 변환하면 안되므로, encoders를 구성할 때 object type columns를 미리 지정해준다.(obj)

In [21]:
# data type set
obj = ['마더코드', '상품코드', '상품군', 'week', 'hour', 'weekofyear']

for col in obj:
    pf_train[col] = pf_train[col].astype('U')

del pf_train['설명']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [22]:
pf_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31841 entries, 15609 to 29649
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   방송일시        31841 non-null  object 
 1   노출(분)       31841 non-null  float64
 2   마더코드        31841 non-null  object 
 3   상품코드        31841 non-null  object 
 4   상품명         31841 non-null  object 
 5   상품군         31841 non-null  object 
 6   판매단가        31841 non-null  int64  
 7   취급액         31841 non-null  float64
 8   date        31841 non-null  object 
 9   week        31841 non-null  object 
 10  time        31841 non-null  object 
 11  hour        31841 non-null  object 
 12  prime       31841 non-null  int64  
 13  real_date   31841 non-null  object 
 14  IsHoliday   31841 non-null  float64
 15  지속휴일수       31841 non-null  float64
 16  TEMP        31841 non-null  float64
 17  HUM         31841 non-null  float64
 18  weekofyear  31841 non-null  object 
 19  stage       31841 non

In [23]:
# Encoding into DECIMAL values
dtypes = pf_train.dtypes
encoders = {}

# 미리 지정한 obj 컬럼들에 대해서만 인코딩 진행
for column in obj:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(pf_train[column])
        encoders[column] = encoder

_data = pf_train.copy()
for column in encoders.keys():
    encoder = encoders[column]
    np.save(column+'_train_classes.npy', encoder.classes_)
    _data[column] = encoder.transform(pf_train[column])

In [ ]:
encoders['마더코드'].classes_

In [24]:
_data.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,hour,prime,real_date,IsHoliday,지속휴일수,TEMP,HUM,weekofyear,stage
15609,2019-06-07 21:40:00,20.0,166,553,(무)[보루네오] 피올레 천연소가죽 소파 3인용,0,749000,5840000.0,2019-06-07 00:00:00,4,21:40:00,14,1,2019-06-07 00:00:00,0.0,0.0,19.221877,77.108243,15,late
25855,2019-09-21 13:40:00,10.0,12,16,AAD 소가죽 올데이슬립온,8,39000,18599000.0,2019-09-21 00:00:00,5,13:40:00,5,1,2019-09-21 00:00:00,1.0,2.0,19.665054,75.075588,31,late


In [ ]:
encoders

In [ ]:
_data['상품코드'].max()

In [ ]:
len(_data)

In [25]:
_data.describe()

,노출(분),마더코드,상품코드,상품군,판매단가,취급액,week,hour,prime,IsHoliday,지속휴일수,TEMP,HUM,weekofyear
count,31841.000000,31841.000000,31841.000000,31841.000000,3.184100e+04,3.184100e+04,31841.000000,31841.000000,31841.000000,31841.000000,31841.000000,31841.000000,31841.000000,31841.000000
mean,20.340387,323.000471,987.485883,5.075940,3.885730e+05,2.313602e+07,3.004083,10.278038,0.351905,0.321410,0.706322,13.936443,62.501395,24.995289
std,3.086391,204.507911,598.064627,3.051538,6.282720e+05,2.003681e+07,2.008880,6.037835,0.477572,0.467025,1.114529,9.633709,18.177993,15.026389
min,2.466667,0.000000,0.000000,0.000000,1.280000e+04,1.030000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,-8.577640,16.928262,0.000000
25%,20.000000,122.000000,445.000000,3.000000,5.900000e+04,8.016000e+06,1.000000,5.000000,0.000000,0.000000,0.000000,5.623889,49.195754,12.000000
50%,20.000000,302.000000,949.000000,5.000000,9.900000e+04,1.738200e+07,3.000000,11.000000,0.000000,0.000000,0.000000,14.784775,63.524958,25.000000
75%,20.000000,492.000000,1504.000000,8.000000,3.990000e+05,3.279300e+07,5.000000,15.000000,1.000000,1.000000,2.000000,22.286536,77.467490,38.000000
max,40.000000,686.000000,2023.000000,10.000000,7.930000e+06,3.220090e+08,6.000000,20.000000,1.000000,1.000000,5.000000,33.703554,96.144207,51.000000


binary encoding을 진행하기 전에 최대값을 이진수로 바꾸었을 때 제일 긴 값의 길이가 몇인지 확인한다.

In [26]:
_data.describe()[obj].loc['max'].apply(lambda x: len(bin(int(x)).replace("0b", "")))

마더코드          10
상품코드          11
상품군            4
week           3
hour           5
weekofyear     6
Name: max, dtype: int64

In [27]:
# categorical variables to binary encoding
# bin(max(_data['상품명'])).replace("0b", '')

def binary_encoding(name, x):
    # x : column
    if name == '마더코드':
        value = bin(x).replace("0b", "")
        return list(map(int, '0'*(10-len(value))+value))
    if name == '상품코드':
        value = bin(x).replace("0b", "")
        return list(map(int, '0'*(11-len(value))+value))
    if name == 'week':
        value = bin(x).replace("0b", "")
        return list(map(int, '0'*(3-len(value))+value))
    if name == 'hour':
        value = bin(x).replace("0b", "")
        return list(map(int, '0'*(5-len(value))+value))
    if name == 'weekofyear':
        value = bin(x).replace("0b", "")
        return list(map(int, '0'*(6-len(value))+value))
    else: # 상품군
        value = bin(x).replace("0b", "")
        return list(map(int, '0'*(4-len(value))+value))
    

In [28]:
# 마더코드
final = _data.copy()
final[['m_'+str(i) for i in range(10)]] = _data['마더코드'].apply(lambda x: binary_encoding('마더코드', x)).to_list()
# final.head(2)

In [29]:
# 상품코드
final[['p_'+str(i) for i in range(11)]] = _data['상품코드'].apply(lambda x: binary_encoding('상품코드', x)).to_list()
# final.head(2)

In [30]:
# 상품군
final[['g_'+str(i) for i in range(4)]] = _data['상품군'].apply(lambda x: binary_encoding('상품군', x)).to_list()
# final.head(2)

In [31]:
# week
final[['wd_'+str(i) for i in range(3)]] = _data['week'].apply(lambda x: binary_encoding('week', x)).to_list()


In [32]:
# weekofyear
final[['wy_'+str(i) for i in range(6)]] = _data['weekofyear'].apply(lambda x: binary_encoding('weekofyear', x)).to_list()
final.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,...,g_3,wd_0,wd_1,wd_2,wy_0,wy_1,wy_2,wy_3,wy_4,wy_5
15609,2019-06-07 21:40:00,20.0,166,553,(무)[보루네오] 피올레 천연소가죽 소파 3인용,0,749000,5840000.0,2019-06-07 00:00:00,4,...,0,1,0,0,0,0,1,1,1,1
25855,2019-09-21 13:40:00,10.0,12,16,AAD 소가죽 올데이슬립온,8,39000,18599000.0,2019-09-21 00:00:00,5,...,0,1,0,1,0,1,1,1,1,1


In [33]:
# hour
final[['h_'+str(i) for i in range(5)]] = _data['hour'].apply(lambda x: binary_encoding('hour', x)).to_list()


In [43]:
def encode_stage(x):
    if x == 'early':
        return 0
    if x == 'mid':
        return 1
    else:
        return 2

In [44]:
final['stage'] = final['stage'].apply(lambda x: encode_stage(x))

In [45]:
final.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,...,wy_1,wy_2,wy_3,wy_4,wy_5,h_0,h_1,h_2,h_3,h_4
15609,2019-06-07 21:40:00,20.0,166,553,(무)[보루네오] 피올레 천연소가죽 소파 3인용,0,749000,5840000.0,2019-06-07 00:00:00,4,...,0,1,1,1,1,0,1,1,1,0
25855,2019-09-21 13:40:00,10.0,12,16,AAD 소가죽 올데이슬립온,8,39000,18599000.0,2019-09-21 00:00:00,5,...,1,1,1,1,1,0,0,1,0,1


In [46]:
final['stage']

15609    2
25855    2
31979    2
12440    1
9664     1
        ..
9077     2
9503     2
18667    2
23556    0
29649    0
Name: stage, Length: 31841, dtype: int64

In [48]:
final.to_csv("encoded_data_ver2.csv", index = False)

In [56]:
train = final.drop(['마더코드', '상품코드', '상품명', '상품군'], axis = 1)
# display(train.head(2))
# train.describe() 

In [57]:
train = train.drop(['방송일시', 'date', 'time', 'real_date'], axis = 1)
train.head(2)

,노출(분),판매단가,취급액,week,hour,prime,IsHoliday,지속휴일수,TEMP,HUM,...,wy_1,wy_2,wy_3,wy_4,wy_5,h_0,h_1,h_2,h_3,h_4
15609,20.0,749000,5840000.0,4,14,1,0.0,0.0,19.221877,77.108243,...,0,1,1,1,1,0,1,1,1,0
25855,10.0,39000,18599000.0,5,5,1,1.0,2.0,19.665054,75.075588,...,1,1,1,1,1,0,0,1,0,1


### Scale (Min-Max Scaler)

In [51]:
# scale dataset
scaler = MinMaxScaler()
cols = ['TEMP', 'HUM']
train[cols] = scaler.fit_transform(train[cols])

In [52]:
train.shape

(31841, 51)

## Modeling

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Embedding, LSTM, concatenate
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
print(tf.__version__)

## Dataset

In [53]:
X = train.drop(['취급액'], axis = 1)
Y = train[['취급액']]
print("data shape : ", X.shape)
print("label shape : " , Y.shape)

data shape :  (31841, 50)
label shape :  (31841, 1)


## XGBoost

In [ ]:
import xgboost as xgb
import seaborn as sns

In [ ]:
xgb.__version__

In [ ]:
model1 = xgb.XGBRegressor(learning_rate = 0.1,
                          max_depth = 20,
                          n_estimators = 100)

In [ ]:
model1.fit(x_train, y_train)

In [ ]:
# y_train = y_train.values.reshape(y_train.shape[0],)
y_valid = y_valid.values.reshape(y_valid.shape[0],)

In [ ]:
sns.scatterplot(y_train, model1.predict(x_train))

In [ ]:
y_pred = model1.predict(x_valid)
sns.scatterplot(y_valid, y_pred)

In [ ]:
model1.score(x_valid, y_valid)

In [41]:
from sklearn.metrics import accuracy_score
from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = check_array(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return ('MAPE', np.mean(np.abs((y_true - y_pred) / y_true)))

In [ ]:
mean_absolute_percentage_error(y_valid, y_pred)

In [ ]:
model2 = xgb.XGBRegressor(learning_rate = 0.1,
                          objective = mean_absolute_percentage_error,
                          max_depth = 20,
                          n_estimators = 100)

In [ ]:
y_train =y_train[:, np.newaxis]
y_train.shape

In [ ]:
model2.fit(x_train, y_train,
          eval_set = [(x_train, y_train), (x_valid, y_valid)])

In [ ]:
model1.save_model("xgb20.json")

## Random Forest

In [58]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth = 100,
                           random_state = 0)
rf.fit(X, Y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestRegressor(max_depth=100, random_state=0)

### Filtering

In [55]:
pf_valid.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,...,hour,prime,real_date,설명,IsHoliday,지속휴일수,TEMP,HUM,weekofyear,stage
3611,2019-02-09 23:20:00,20.0,100585,201801,임페리얼 여성 울 블렌딩 니트 3종,의류,49000,16797000.0,2019-02-09 00:00:00,5,...,23,0,2019-02-09 00:00:00,NaN,1.0,2.0,-4.328142,36.256716,6,mid
27653,2019-10-09 10:40:00,20.0,100753,202202,아가타 골든 마스터 2종 (펌프스1종+플랫슈즈1종),잡화,59900,82482000.0,2019-10-09 00:00:00,2,...,10,0,2019-10-09 00:00:00,한글날,1.0,1.0,14.708759,50.103458,41,late


In [59]:
# data type set
for col in obj:
    pf_valid[col] = pf_valid[col].astype('U')

del pf_valid['설명']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
pf_valid['stage'] = pf_valid['stage'].apply(lambda x: encode_stage(x))
valid = pf_valid.drop(['마더코드', '상품코드', '상품명', '상품군',
                      '방송일시', 'date', 'time', 'real_date'], axis = 1)
valid.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,노출(분),판매단가,취급액,week,hour,prime,IsHoliday,지속휴일수,TEMP,HUM,weekofyear,stage
3611,20.0,49000,16797000.0,5,23,0,1.0,2.0,-4.328142,36.256716,6,1
27653,20.0,59900,82482000.0,2,10,0,1.0,1.0,14.708759,50.103458,41,2
21684,20.0,79000,14795000.0,4,10,0,0.0,0.0,29.186814,70.736485,32,0
25128,20.0,55900,55171000.0,5,16,1,1.0,4.0,24.353742,76.226286,37,2
16944,20.0,89000,23745000.0,4,23,0,0.0,0.0,19.769987,81.130585,25,2


In [63]:
m_classes = np.load('마더코드_train_classes.npy')
p_classes = np.load('상품코드_train_classes.npy')
# s_classes = np.load('reference/stage_classes.npy')
g_classes = np.load('상품군_train_classes.npy')

In [ ]:
final = []
values = valid.values
for i in range(len(values)):
    # 상품코드가 인코더 안에 있는지 확인
    if values[i][1] in p_classes:
        final.append(values[i])
    else:
        # 마더코드가 인코더 안에 있는지 확인
        if values[i][0] in m_classes:
            temp = train.loc[train['마더코드'] == np.where(m_classes == values[i][1])[0][0]]
            # 판매단가 차가 최소인 row를 추가함.
            temp['sub'] = abs(temp['판매단가'] - values[i][3])
            final.append(train.iloc[temp['sub'].idxmin()].values)
        else:
            if values[i][2] in g_classes:
                temp = train.loc[train['상품군'] == np.where(g_classes == values[i][2])[0][0]]
                # 판매단가 차가 최소인 row를 추가함.
                temp['sub'] = abs(temp['판매단가'] - values[i][3])
                final.append(train.iloc[temp['sub'].idxmin()].values)

In [ ]:
# week
final[['wd_'+str(i) for i in range(3)]] = _data['week'].apply(lambda x: binary_encoding('week', x)).to_list()

# weekofyear
valid[['wy_'+str(i) for i in range(6)]] = _data['weekofyear'].apply(lambda x: binary_encoding('weekofyear', x)).to_list()
final.head(2)

In [ ]:
rf_pred = rf.predict(x_valid)
mean_absolute_percentage_error(y_valid, rf_pred)

In [ ]:
sns.scatterplot(y_valid, rf_pred)

In [ ]:
rf.score(x_train, y_train)

In [ ]:
rf.score(x_valid, y_valid)

## stack 3 RF models

In [ ]:
rf_1 = RandomForestRegressor(max_depth = 30,
                           random_state = 1)
rf_1.fit(x_train, y_train)

In [ ]:
rf_2 = RandomForestRegressor(max_depth = 20,
                           random_state = 0)
rf_2.fit(x_train, y_train)

In [ ]:
rf1_pred = rf_1.predict(x_valid)
rf2_pred = rf_2.predict(x_valid)

In [ ]:
rf_3 = (rf_pred+rf1_pred+rf2_pred) / 3
rf_3

In [ ]:
mean_absolute_percentage_error(y_valid, rf_3)

In [ ]:
sns.scatterplot(y_valid, rf_3)

## LightGBM